# Overview
- EDA

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from plotly import express as px
from pathlib import Path
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

## Config

In [2]:
class CFG:
    INPUT_DIR = "../input/mlb-player-digital-engagement-forecasting"
    # INPUT_DIR = "../input/mlb-unnested-dataset" # for kaggle kernel
    OBJECT_ID = ["playerId", "date"]
    TARGETS = ["target1", "target2", "target3", "target4"]

    # training
    ## LightGBM
    LGB_PARAMS = {
        "objective": "regression",
        "metric": "mae",
        "boosting_type": "gbdt",
        "learning_rate": 0.01,
        "max_depth": 6,
        "num_leaves": int(64 * 0.8),
        "lambda_l1": 0.001,
        "lambda_l2": 0.001,
        "bagging_fraction": 0.9,
        "bagging_freq": 3,
        "feature_fraction": 0.9,
        "min_data_in_leaf": 20,
        "num_threads": 8,
        "verbosity": -1,
        "num_iterations": 10000,
        "early_stopping_round": 100,
    }
    SEEDS = [2434, 42]

## Load dataset

In [3]:
class MLBDataset(object):
    def __init__(self, input_path):
        self.input_path = Path(input_path)
        #################
        # train #
        #################
        self.train_next = pd.read_pickle(self.input_path / "train_nextDayPlayerEngagement.pickle")
        self.train_rosters = pd.read_pickle(self.input_path / "train_rosters.pickle")
        self.train_ptf = pd.read_pickle(self.input_path / "train_playerTwitterFollowers.pickle")
        self.train_scores = pd.read_pickle(self.input_path / "train_playerBoxScores.pickle")
        self.train_games = pd.read_pickle(self.input_path / "train_games.pickle")
        self.train_standings = pd.read_pickle(self.input_path / "train_standings.pickle")
        self.train_tbs = pd.read_pickle(self.input_path / "train_teamBoxScores.pickle")
        self.train_ttf = pd.read_pickle(self.input_path / "train_teamTwitterFollowers.pickle")
        self.train_trans = pd.read_pickle(self.input_path / "train_transactions.pickle")
        self.train_awards = pd.read_pickle(self.input_path / "train_awards.pickle")
        self.train_events = pd.read_pickle(self.input_path / "train_events.pickle")
        #################
        # Additional #
        #################
        self.players = pd.read_pickle(self.input_path / "players.pickle")
        self.awards = pd.read_pickle(self.input_path / "awards.pickle")
        self.seasons = pd.read_pickle(self.input_path / "seasons.pickle")
        self.teams = pd.read_pickle(self.input_path / "teams.pickle")
        #################
        # test #
        #################
        self.example_test = pd.read_csv(self.input_path / "example_test.csv")
        self.sample_submission = pd.read_csv(self.input_path / "example_sample_submission.csv")

ds = MLBDataset(CFG.INPUT_DIR)

## train_next

In [4]:
ds.train_next.head()

,date,engagementMetricsDate,playerId,target1,target2,target3,target4
0,20180101,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294
1,20180101,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118
2,20180101,2018-01-02,519317,0.974327,56.177044,13.693746,64.166664
3,20180101,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745
4,20180101,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373


- engagementMetricsDate - 米国太平洋時間に基づくプレーヤーエンゲージメント指標の日付（前日のゲーム、名簿、フィールド統計、トランザクション、賞などと一致します）。
- playerId: 選手のユニークID
- target1
- target2
- target3
- target4
- target1-target4は、0から100のスケールでの日毎のデジタルエンゲージメント

## train_games

In [5]:
ds.train_games.head()

,date,gamePk,gameType,season,gameDate,gameTimeUTC,resumeDate,resumedFrom,codedGameState,detailedGameState,isTie,gameNumber,doubleHeader,dayNight,scheduledInnings,gamesInSeries,seriesDescription,homeId,homeName,homeAbbrev,homeWins,homeLosses,homeWinPct,homeWinner,homeScore,awayId,awayName,awayAbbrev,awayWins,awayLosses,awayWinPct,awayWinner,awayScore
0,20180221,533782,E,2018,2018-02-21,2018-02-21T20:10:00Z,NaN,NaN,F,Final,0.0,1,N,day,7,0.0,Exhibition,109,Arizona Diamondbacks,ARI,1,0,1.0,True,7.0,5035,Arizona State Sun Devils,ASU,0.0,1.0,0.0,False,2.0
1,20180222,534461,E,2018,2018-02-22,2018-02-22T18:05:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,0.0,Exhibition,116,Detroit Tigers,DET,1,0,1.0,True,6.0,228,Florida Southern College Mocs,FSC,0.0,1.0,0.0,False,1.0
2,20180222,545334,E,2018,2018-02-22,2018-02-22T18:05:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,0.0,Exhibition,143,Philadelphia Phillies,PHI,1,0,1.0,True,6.0,231,University of Tampa Spartans,UT,0.0,1.0,0.0,False,0.0
3,20180222,547295,E,2018,2018-02-22,2018-02-22T03:33:00Z,NaN,NaN,F,Final,0.0,2,Y,night,9,0.0,Exhibition,111,Boston Red Sox,BOS,2,0,1.0,True,4.0,227,Boston College Eagles,BC,0.0,1.0,0.0,False,2.0
4,20180222,533784,E,2018,2018-02-22,2018-02-22T23:05:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,0.0,Exhibition,142,Minnesota Twins,MIN,1,0,1.0,True,2.0,4864,Minnesota Gophers,UM,0.0,1.0,0.0,False,1.0


- gamePk  : ゲームの一意の識別子。
- gameType  : ゲームの種類、さまざまな種類がここにあります。
    - [{"id":"S","description":"Spring Training"},{"id":"R","description":"Regular Season"},{"id":"F","description":"Wild Card Game"},{"id":"D","description":"Division Series"},{"id":"L","description":"League Championship Series"},{"id":"W","description":"World Series"},{"id":"C","description":"Championship"},{"id":"N","description":"Nineteenth Century Series"},{"id":"P","description":"Playoffs"},{"id":"A","description":"All-Star Game"},{"id":"I","description":"Intrasquad"},{"id":"E","description":"Exhibition"}]
- season :
- gameDate :
- gameTimeUTC  : UTCでの始球式。
- resumeDate  : タイムゲームが再開されました（放棄された場合、それ以外の場合はnull）。
- resumedFrom  : タイムゲームは元々放棄されていました（放棄された場合、それ以外の場合はnull）。
- codedGameState  : ゲームのステータスコード、さまざまなタイプがここにあります。
- detailedGameState  : ゲームのステータス、さまざまな種類がここにあります。
- isTie  : ブール値。ゲームが引き分けで終了した場合はtrue。
- gameNumber  : ダブルヘッダーを区別するためのゲーム番号フラグ
- doubleHeader  : YはDH、Nはシングルゲーム、Sはスプリット
- dayNight  : スケジュールされた開始時間の昼または夜のフラグ。
- scheduledInnings  : 予定イニング数。
- gamesInSeries  : 現在のシリーズのゲーム数。
- seriesDescription  : 現在のシリーズのテキスト説明。
- homeId  : ホームチームの一意の識別子。
- homeName  : ホームチーム名。
- homeAbbrev  : ホームチームの略語。
- homeWins  : ホームチームのシーズンの現在の勝利数。
- homeLosses  : ホームチームのシーズンでの現在の損失数。
- homeWinPct  : ホームチームの現在の勝率。
- homeWinner  : ブール値。ホームチームが勝った場合はtrue。
- homeScore  : ホームチームが得点するラン。
- awayId  : アウェイチームの一意の識別子。
- awayName  : アウェイチームの一意の識別子。
- awayAbbrev  : アウェイチームの略。
- awayWins  : アウェイチームのシーズンの現在の勝利数。
- awayLosses  : アウェイチームのシーズン中の現在の敗北数。
- awayWinPct  : アウェイチームの現在の勝率。
- awayWinner  : ブール値。離れたチームが勝った場合はtrue。
- awayScore  : アウェイチームが得点したラン。

## train_rosters

In [6]:
ds.train_rosters.head()

,date,playerId,gameDate,teamId,statusCode,status
0,20180101,400121,2018-01-01,116,A,Active
1,20180101,408045,2018-01-01,142,A,Active
2,20180101,425492,2018-01-01,120,A,Active
3,20180101,429664,2018-01-01,136,A,Active
4,20180101,431151,2018-01-01,121,A,Active


- playerId: プレーヤーの一意の識別子。
- gameDate: 試合日時 
- teamI: -そのプレーヤーがその日にいるteamId。
- statusCode: 名簿ステータスの略語。
- status: 説明的な名簿のステータス。

## train_playerBoxScores

In [7]:
ds.train_scores[ds.train_scores["playerName"] == "Shohei Ohtani"].head(10).T

,228,1475,1901,2446,2991,4003,4529,4822,5322,5560
date,20180329,20180401,20180403,20180404,20180406,20180408,20180410,20180411,20180412,20180413
home,0,0,1,1,1,1,0,0,0,0
gamePk,529412,529450,529475,529486,529514,529542,529565,529579,529592,529602
gameDate,2018-03-29,2018-04-01,2018-04-03,2018-04-04,2018-04-06,2018-04-08,2018-04-10,2018-04-11,2018-04-12,2018-04-13
gameTimeUTC,2018-03-29T20:05:00Z,2018-04-01T20:05:00Z,2018-04-04T02:07:00Z,2018-04-04T20:07:00Z,2018-04-07T02:07:00Z,2018-04-08T20:07:00Z,2018-04-11T00:05:00Z,2018-04-12T00:05:00Z,2018-04-13T00:15:00Z,2018-04-14T00:15:00Z
teamId,108,108,108,108,108,108,108,108,108,108
teamName,Los Angeles Angels,Los Angeles Angels,Los Angeles Angels,Los Angeles Angels,Los Angeles Angels,Los Angeles Angels,Los Angeles Angels,Los Angeles Angels,Los Angeles Angels,Los Angeles Angels
playerId,660271,660271,660271,660271,660271,660271,660271,660271,660271,660271
playerName,Shohei Ohtani,Shohei Ohtani,Shohei Ohtani,Shohei Ohtani,Shohei Ohtani,Shohei Ohtani,Shohei Ohtani,Shohei Ohtani,Shohei Ohtani,Shohei Ohtani
jerseyNum,17,17,17,17,17,17,17,17,17,17


- home  : バイナリ、ホームチームの場合は1、離れている場合は0。
- gamePk  : ゲームの一意の識別子。
- gameDate :
- gameTimeUTC  : UTCでの始球式。
- teamId  : チームの一意の識別子。
- teamName :
- playerId  : プレーヤーの一意の識別子。
- playerName :
- jerseyNum : 背番号
- positionCode  : 番号の位置コード、詳細はこちらです。
- positionName  : テキスト位置の表示、詳細はこちらです。
- positionType  : ポジショングループ、詳細はこちらです。
- battingOrder  : 形式：「###」。最初の桁は打順スポットを示し、次の2桁はそのプレーヤーがその打順スポットを占めた順序を示します。例：「300」は、打順の3番目のスポットのスターターを示します。 4人目（900、901、902以降）が打順9位を占めることを示す「903」。ゲームに登場した場合にのみ入力されます。
- gamesPlayedBatting  : プレーヤーが打者、ランナー、または野手としてゲームに参加した場合は1。
- flyOuts  : ゲームの合計フライアウト。
- groundOuts  : ゲームのトータルグラウンドアウト。
- runsScored  : ゲームの合計ランが記録されました。
- doubles  : ゲームの合計二塁打。
- triples  : ゲームの合計三塁打。
- homeRuns  : ゲームの総本塁打。
- strikeOuts  : ゲームの合計三振。
- baseOnBalls  : ゲームの合計四球数。
- intentionalWalks  : ゲームの故意四球。
- hits  : ゲームの総ヒット数。
- hitByPitch  : 死球数。
- atBats  : ゲームの打数。
- caughtStealing  : ゲームの合計が盗塁をキャッチしました。
- stolenBases  : ゲームの盗塁総数。
- groundIntoDoublePlay  : ゲームの合計併殺。
- groundIntoTriplePlay  : ゲームの合計トリプルプレイ。
- plateAppearances  : ゲームの打席数。
- totalBases  : ゲームのトータル塁打数。
- rbi  : ゲームの合計打点。
- leftOnBase  : ゲームの総残塁数。
- sacBunts  : ゲームの合計犠牲バント。
- sacFlies  : ゲームの総犠牲フライ。
- catchersInterference  : ゲームのトータルキャッチャーの干渉が発生しました。
- pickoffs  : ゲームの合計回数がベースから外れました。
- gamesPlayedPitching : バイナリ、プレーヤーが投手としてゲームに参加した場合は 1。
- gamesStartedPitching : バイナリ、プレーヤーがゲームの先発投手だった場合は1。
- completeGamesPitching  : バイナリ、完投でクレジットされている場合は1。
- shutoutsPitching  : バイナリ、完封でクレジットされている場合は1。
- winsPitching  : バイナリ、勝利でクレジットされている場合は 1。
- lossesPitching  : バイナリ、損失がクレジットされている場合は1。
- flyOutsPitching  : 許可されたフライアウトのゲーム合計。
- airOutsPitching  : エアアウト（フライアウト+ポップアウト）のゲーム合計が許可されます。
- groundOutsPitching  : ゲームの合計グラウンドアウトが許可されます。
- runsPitching  : ゲームの合計実行が許可されます。
- doublesPitching  : ゲームの合計は2倍になります。
- triplesPitching  : ゲームの合計トリプルが許可されます。
- homeRunsPitching  : ゲームの合計ホームランが許可されます。
- strikeOutsPitching  : ゲームの合計三振が許可されます。
- baseOnBallsPitching  : ゲームの合計歩行が許可されます。
- intentionalWalksPitching  : ゲームの故意四球の合計が許可されます。
- hitsPitching  : 許可されるゲームの合計ヒット数。
- hitByPitchPitching  : 許可されたピッチによるゲームの合計ヒット。
- atBatsPitching  : でのゲーム合計
- caughtStealingPitching  : ゲームの合計は、盗みをキャッチしました。
- stolenBasesPitching  : ゲームの盗塁の合計は許可されます。
- inningsPitched  : ゲームの総投球回。
- saveOpportunities  : バイナリ、保存の機会がある場合は1。
- earnedRuns  : ゲームの合計自責点は許可されています。
- battersFaced  : 直面したゲームの総打者。
- outsPitching  : ゲームの合計アウトが記録されました。
- pitchesThrown  : 投げられた投球のゲーム総数。
- balls  : 投げられたゲームの合計ボール。
- strikes  : スローされたゲームの合計ストライク。
- hitBatsmen  : ゲームの総死球打者。
- balks  : ゲームの合計はボークします。
- wildPitches  : 投げられた暴投のゲーム総数。
- pickoffsPitching  : ゲームのピックオフの総数。
- rbiPitching  : 打点のゲーム総数は許可されています。
- inheritedRunners  : 継承されたランナーのゲーム合計を想定。
- inheritedRunnersScored : 得点した継承されたランナーのゲーム合計。
- catchersInterferencePitching  : キャッチャーの干渉のゲーム合計はバッテリーによって発生しました。
- sacBuntsPitching  : ゲームの犠牲バントの合計が許可されます。
- sacFliesPitching  : ゲームの犠牲フライは許可されています。
- saves  : バイナリ、保存でクレジットされている場合は1。
- holds  : バイナリ、保留がクレジットされている場合は1。
- blownSaves  : バイナリ、ブローセーブでクレジットされている場合は1。
- assists  : ゲームのアシスト総数。
- putOuts  : ゲームの刺殺の総数。
- errors  : ゲームのエラーの総数。
- chances  : ゲームのトータルフィールディングチャンス。

## train_teamBoxScores

In [8]:
ds.train_tbs.head().T

,0,1,2,3,4
date,20180329,20180329,20180329,20180329,20180329
home,1,0,1,1,1
teamId,109,114,121,139,140
gamePk,529410,529409,529419,529406,529411
gameDate,2018-03-29,2018-03-29,2018-03-29,2018-03-29,2018-03-29
gameTimeUTC,2018-03-30T02:10:00Z,2018-03-30T02:10:00Z,2018-03-29T17:10:00Z,2018-03-29T20:00:00Z,2018-03-29T19:35:00Z
flyOuts,4,4,2,2,9
groundOuts,9,9,10,6,4
runsScored,8,1,9,6,1
doubles,2,1,2,1,1


- home  : バイナリ、ホームチームの場合は1、離れている場合は0。
- teamId  : チームの一意の識別子。
- gamePk  : ゲームの一意の識別子。
- gameDate :
- gameTimeUTC  : UTCでの始球式。
- flyOuts  : ゲームの合計フライアウト。
- groundOuts  : ゲームのトータルグラウンドアウト。
- runsScored  : ゲームの合計ランが記録されました。
- doubles  : ゲームの合計は2倍です。
- triples  : ゲームの合計トリプル。
- homeRuns  : ゲームの総本塁打。
- strikeOuts  : ゲームの合計三振。
- baseOnBalls  : ゲームの合計ウォーク。
- intentionalWalks  : ゲームの故意四球。
- hits  : ゲームの総ヒット数。
- hitByPitch  : ピッチによるゲームの合計ヒット。
- atBats  : でのゲーム合計
- caughtStealing  : ゲームの合計が盗塁をキャッチしました。
- stolenBases  : ゲームの盗塁総数。
- groundIntoDoublePlay  : ゲームの合計併殺はに基づいています。
- groundIntoTriplePlay  : ゲームの合計 3 回プレイが基礎になります。
- plateAppearances  : ゲームの総打席。
- totalBases  : ゲームの総拠点数。
- rbi  : ゲームの合計打点。
- leftOnBase  : ゲームの総ランナーはベースに残った。
- sacBunts  : ゲームの合計犠牲バント。
- sacFlies  : ゲームの総犠牲フライ。
- catchersInterference  : ゲームのトータルキャッチャーの干渉が発生しました。
- pickoffs  : ゲームの合計回数がベースから外れました。
- airOutsPitching  : エアアウト（フライアウト+ポップアウト）のゲーム合計が許可されます。
- groundOutsPitching  : ゲームの合計グラウンドアウトが許可されます。
- runsPitching  : ゲームの合計実行が許可されます。
- doublesPitching  : ゲームの合計は2倍になります。
- triplesPitching  : ゲームの合計トリプルが許可されます。
- homeRunsPitching  : ゲームの合計ホームランが許可されます。
- strikeOutsPitching  : ゲームの合計三振が許可されます。
- baseOnBallsPitching  : ゲームの合計歩行が許可されます。
- intentionalWalksPitching  : ゲームの故意四球の合計が許可されます。
- hitsPitching  : 許可されるゲームの合計ヒット数。
- hitByPitchPitching  : 許可されたピッチによるゲームの合計ヒット。
- atBatsPitching  : でのゲーム合計
- caughtStealingPitching  : ゲームの合計は、盗みをキャッチしました。
- stolenBasesPitching  : ゲームの盗塁の合計は許可されます。
- inningsPitched  : ゲームの総投球回。
- earnedRuns  : ゲームの合計自責点は許可されています。
- battersFaced  : 直面したゲームの総打者。
- outsPitching  : ゲームの合計アウトが記録されました。
- hitBatsmen  : ゲームの総死球打者。
- balks  : ゲームの合計はボークします。
- wildPitches  : 投げられた暴投のゲーム総数。
- pickoffsPitching  : ゲームのピックオフの総数。
- rbiPitching  : 打点のゲーム総数は許可されています。
- inheritedRunners  : 継承されたランナーのゲーム合計を想定。
- inheritedRunnersScored : 得点した継承されたランナーのゲーム合計。
- catchersInterferencePitching  : キャッチャーの干渉のゲーム合計はバッテリーによって発生しました。
- sacBuntsPitching  : ゲームの犠牲バントの合計が許可されます。
- sacFliesPitching  : ゲームの犠牲フライは許可されています。

## train_transactions

In [9]:
ds.train_trans.head().T

,0,1,2,3,4
date,20180101,20180102,20180102,20180102,20180102
transactionId,340732,339458,357292,341123,339458
playerId,547348.0,621173.0,678876.0,607054.0,NaN
playerName,C.C. Lee,Dylan Baker,Angel Rojas,Jace Peterson,None
fromTeamId,NaN,158.0,NaN,NaN,119.0
fromTeamName,NaN,Milwaukee Brewers,None,None,Los Angeles Dodgers
toTeamId,119,119,147,147,158
toTeamName,Los Angeles Dodgers,Los Angeles Dodgers,New York Yankees,New York Yankees,Milwaukee Brewers
effectiveDate,2018-01-01,2018-03-20,2018-01-02,2018-01-02,2018-01-02
resolutionDate,2018-01-01,None,2018-01-02,2018-01-02,None


- transactionId  : トランザクションの一意の識別子。
- playerId  : プレーヤーの一意の識別子。
- playerName :
- date :
- fromTeamId  : プレーヤーの出身チームの一意の識別子。
- fromTeamName :
- toTeamId  : プレーヤーが行くチームの一意の識別子。
- toTeamName :
- effectiveDate :
- resolutionDate :
- typeCode  : トランザクションステータスの略語。
- typeDesc  : トランザクションステータスの説明。
- description  : トランザクションのテキスト説明。

## train_standings

In [10]:
ds.train_standings.head().T

,0,1,2,3,4
date,20180329,20180329,20180329,20180329,20180329
season,2018,2018,2018,2018,2018
gameDate,2018-03-29,2018-03-29,2018-03-29,2018-03-29,2018-03-29
divisionId,205,204,204,200,204
teamId,112,146,121,140,144
teamName,Chicago Cubs,Miami Marlins,New York Mets,Texas Rangers,Atlanta Braves
streakCode,W1,L1,W1,L1,W1
divisionRank,1,4,2,5,1
leagueRank,3,12,5,14,2
wildCardRank,3.0,12.0,5.0,13.0,2.0


- season :
- gameDate :
- divisionId  : このチームが所属する部門を表す一意識別子。
- teamId  : チームの一意の識別子。
- teamName :
- streakCode  : チームの現在の勝ち負けの連続の略語。最初の文字は勝ち負けを示し、数字はゲームの数です。
- divisionRank  : チームの部門における現在のランク。
- leagueRank  : リーグでのチームの現在のランク。
- wildCardRank  : ワイルドカードバースのチームの現在のランク。
- leagueGamesBack  : ゲームはチームのリーグに戻ります。
- sportGamesBack  : MLBのすべてに戻ってゲーム。
- divisionGamesBack  : チームの部門にゲームが戻ってきました。
- wins  : 現在の勝利。
- losses  : 現在の損失。
- pct  : 現在の勝率。
- runsAllowed  : シーズン中に許可された実行。
- runsScored  : シーズンに得点したラン。
- divisionChamp  : チームが部門タイトルを獲得した場合はtrue。
- divisionLeader  : チームがディビジョンレースをリードしている場合はtrue。
- wildCardLeader  : チームがワイルドカードリーダーの場合はtrue。
- eliminationNumber  : ディビジョンレースから排除されるまでのゲーム数（チームの敗北+対戦相手の勝利）。
- wildCardEliminationNumber  : ワイルドカードレースから排除されるまでのゲーム数（チームの敗北+対戦相手の勝利）。
- homeWins  : ホームはシーズンに勝ちます。
- homeLosses  : シーズン中のホームロス。
- awayWins  : アウェイはシーズンに勝ちます。
- awayLosses  : シーズンのアウェイロス。
- lastTenWins  : 過去10試合で勝ちました。
- lastTenLosses  : 過去10試合で負けました。
- extraInningWins  : シーズンの追加イニングで勝ちます。
- extraInningLosses  : シーズンの追加イニングでの損失。
- oneRunWins  : シーズン中に1ランで勝ちます。
- oneRunLosses  : シーズン中に1ランで負けます。
- dayWins  : デイゲームはシーズンに勝ちます。
- dayLosses Day game losses on the season. :
- nightWins  : ナイトゲームはシーズンに勝ちます。
- nightLosses  : シーズン中のナイトゲームの敗北。
- grassWins  : 芝生のフィールドがシーズンに勝ちます。
- grassLosses  : 季節の草地の損失。
- turfWins  : 芝フィールドはシーズンに勝ちます。
- turfLosses  : シーズン中の芝フィールドの損失。
- divWins  : シーズン中にディビジョンの対戦相手に勝ちます。
- divLosses  : シーズン中のディビジョンの対戦相手に対する敗北。
- alWins  : シーズン中にALチームに勝ちます。
- alLosses  : シーズン中のALチームに対する敗北。
- nlWins  : シーズン中にNLチームに勝ちます。
- nlLosses  : シーズン中のNLチームに対する敗北。
- xWinLossPct  : スコアリングおよび許可されたランに基づく予想勝率.

## train_awards

In [11]:
ds.train_awards.head()

,date,awardId,awardName,awardDate,awardSeason,playerId,playerName,awardPlayerTeamId
0,20180115,VWLCPOY,VWL Comeback Player of the Year,2018-01-15,2017,150119,Freddy Garcia,699.0
1,20180115,VWLPOY,VWL Pitcher of the Year,2018-01-15,2017,446861,Guillermo Moscoso,699.0
2,20180115,VWLMOY,VWL Manager of the Year,2018-01-15,2017,492527,Mike Rojas,695.0
3,20180115,VWLRLOY,VWL Reliever of the Year,2018-01-15,2017,519246,Andres Santiago,694.0
4,20180115,VWLOPOY,VWL Offensive Player of the Year,2018-01-15,2017,543874,Jose Vargas,699.0


- awardId :
- awardName :
- awardDate  : 日付賞が与えられました。
- awardSeason  : シーズンアワードはからでした。
- playerId  : プレーヤーの一意の識別子。
- playerName :
- awardPlayerTeamId :

## train_events

In [12]:
ds.train_events[(ds.train_events["hitterName"] == "Shohei Ohtani") & (ds.train_events["gamePk"] == 529412)].sort_values("inning").head(10).T

,1754,1629,1872,1199,2055,2412,3546,3976,1882,1291
date,20180329,20180329,20180329,20180329,20180329,20180329,20180329,20180329,20180329,20180329
gamePk,529412,529412,529412,529412,529412,529412,529412,529412,529412,529412
gameDate,2018-03-29,2018-03-29,2018-03-29,2018-03-29,2018-03-29,2018-03-29,2018-03-29,2018-03-29,2018-03-29,2018-03-29
gameTimeUTC,2018-03-29T20:05:00Z,2018-03-29T20:05:00Z,2018-03-29T20:05:00Z,2018-03-29T20:05:00Z,2018-03-29T20:05:00Z,2018-03-29T20:05:00Z,2018-03-29T20:05:00Z,2018-03-29T20:05:00Z,2018-03-29T20:05:00Z,2018-03-29T20:05:00Z
season,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018
gameType,R,R,R,R,R,R,R,R,R,R
playId,b6c61ee6-2fb8-46b2-8fcf-937eed0bab13,31c9bd1d-2bdf-480d-aac8-825bb45839ff,f03e7197-aed6-4325-933e-097ee604a2b7,0cb773ca-9548-471e-9fba-31d61aa1e399,b570bd67-bb56-4e20-af32-2f49de3a7170,fedd84c0-e0ba-4655-8ce1-74f941aa494d,3584f522-8783-4109-a645-b51e3e02996b,c9a16ee0-be76-42cd-9fb5-1e15b324b88b,968bce66-a6e5-4cf8-a10f-89a41d607266,e9cf47b7-5768-4d07-8f9a-23958ab64c2c
eventId,0,0,1,0,1,4,2,5,0,0
inning,2,4,4,6,6,6,6,6,9,11
halfInning,top,top,top,top,top,top,top,top,top,top


- gamePk  : ゲームの一意の識別子。
- gameDate :
- gameTimeUTC  : UTCでの始球式。
- season :
- gameType  : ゲームの種類、さまざまな種類がここにあります。
- playId  : スタットキャストのプレイガイド。
- eventId :
- inning  : イニングABが発生しました。
- halfInning : 「上」または「下」のイニングインジケーター。
- homeScore  : イベント開始時のホームスコア。
- awayScore  : イベント開始時のアウェイスコア。
- menOnBase  : 走者がベースにいる場合に使用されるスプリット–すなわち（RISP、空）。
- atBatIndex  : で
- atBatDesc  : 演奏する
- atBatEvent  : atBatのイベントタイプの結果。さまざまなタイプがここにあります。
- hasOut  : バイナリ、ランナーが場に出ている場合は1。
- pitcherTeamId  : ピッチングチームの一意の識別子。
- isPitcherHome  : バイナリ、投手がホームチームの場合は1。
- pitcherTeam  : ピッチングチームのチーム名。
- hitterTeamId  : 打撃チームの一意の識別子。
- hitterTeam  : 打撃チームのチーム名。
- pitcherId :
- pitcherName :
- isStarter  : バイナリ、プレーヤーがゲームの先発投手だった場合は1。
- pitcherHand  : プレーヤーが手を投げる：「L」、「R」。
- hitterId :
- hitterName :
- batSide  : プレーヤーのバット側：「L」、「R」。
- pitchNumber  : ABのピッチシーケンス番号。
- balls  : イベント後のボール数。
- strikes  : イベント後のストライクカウント。
- isGB  : バイナリ、打席がグラウンドボールの場合は1。
- isLD  : バイナリ、打席がラインドライブの場合は1。
- isFB  : バイナリ、打席が飛球の場合は1。
- isPU  : バイナリ、打席がポップアップの場合は1。
- launchSpeed  : 打球の測定速度。
- launchAngle  : ヒットが開始された地平線に対する垂直角度。
- totalDistance  : ボールが移動した合計距離。
- event  : で発生する可能性のあるイベント
- description  : イベントのテキスト説明。
- rbi  : AB中に打点を打った回数。
- pitchType  : ピッチタイプ分類コード。さまざまなタイプがここにあります。
- call  : 投球または投球の結果分類コード。さまざまなタイプがここにあります。
- outs  : ABの現在/最終アウト。
- inPlay  : ボールが場に出た場合は真/偽。
- isPaOver  : バイナリ、このイベントがプレートの外観の終わりである場合は1。
- startSpeed  : ホームプレートの前50フィートでのボールのMPHでの速度。
- endSpeed  : ボールがホームプレートの前端（x軸で0,0）を横切るときのボールのMPHでの速度。
- nastyFactor  : 各ピッチのいくつかのプロパティを評価し、ピッチの「不快感」を0からのスケールで評価します
- breakAngle  : ピッチの平面が垂直から外れる時計回り（打者の視点）の角度。
- breakLength  : ピッチがピッチ開始とピッチ終了の間の直線から離れる最大距離。
- breakY  : ブレークが最大のホームプレートからの距離。
- spinRate  : ピッチャーによってRPMでリリースされた後のボールのスピン率。
- spinDirection  : スピンがボールの弾道にどのように影響するかを反映する角度として与えられる、リリース時のボールの回転軸。ピュアバック
- pX  : ボールがホームプレートの前軸と交差するときのボールのフィート単位の水平位置。
- pZ  : ボールがホームプレートの前軸と交差するときの、ボールのホームプレートからのフィート単位の垂直位置。
- X  : z軸のボール加速度。
- aY  : y軸のボール加速度。
- aZ  : z 軸上のボールの加速度。
- pfxX  : インチ単位のボールの水平方向の動き。
- pfxZ  : インチ単位のボールの垂直方向の動き。
- vX0  : x軸からのボールの速度。
- vY0  : y軸からのボールの速度。 0,0,0 はバッターの後ろにあり、ボールはピッチャー マウンドから 0,0,0 に向かって移動するため、これは負です。
- vZ0  : z軸からのボールの速度。
- x  : ピッチがホームプレートの前を横切ったX座標。
- y  : ピッチがホームプレートの前面と交差するY座標。
- x0  : ピッチャーの手を離したときのボールの x 軸上の座標位置 (時間 = 0)。
- y0  : y軸上でピッチャーの手からボールがリリースされたポイントでのボールの座標位置（時間= 0）。
- z0  : z軸上でピッチャーの手からボールがリリースされたポイントでのボールの座標位置（時間= 0）。
- type  : 「ピッチ」または「アクション」のいずれかのイベントのタイプ
- zone  : ゾーンロケーション番号.下を参照

## train_playerTwitterFollowers

In [13]:
ds.train_ptf.head()

,date,playerId,playerName,accountName,twitterHandle,numberOfFollowers
0,20180101,545361,Mike Trout,Mike Trout,@miketrout,2452409
1,20180101,506433,Yu Darvish,Yu Darvish,@faridyu,1945081
2,20180101,434378,Justin Verlander,Justin Verlander,@justinverlander,1795985
3,20180101,430897,Nick Swisher,Nick Swisher,@nickswisher,1711807
4,20180101,120074,David Ortiz,David Ortiz,@davidortiz,1515463


- date  : フォロワー数の日付。
- playerId  : プレーヤーの一意の識別子。
- playerName :
- accountName  : プレイヤーのTwitterアカウントの名前。
- twitterHandle  : プレイヤーのツイッターハンドル。
- numberOfFollowers  : フォロワー数

## train_teamTwitterFollowers

In [14]:
ds.train_ttf.head()

,date,teamId,teamName,accountName,twitterHandle,numberOfFollowers
0,20180101,147,New York Yankees,New York Yankees,@Yankees,3130482
1,20180101,112,Chicago Cubs,Chicago Cubs,@Cubs,2373710
2,20180101,141,Toronto Blue Jays,Toronto Blue Jays,@BlueJays,2196352
3,20180101,111,Boston Red Sox,Boston Red Sox,@RedSox,1950737
4,20180101,119,Los Angeles Dodgers,Los Angeles Dodgers,@Dodgers,1949542


- date  : フォロワー数の日付。
- teamId  : チームの一意の識別子。
- teamName :
- accountName  : チームのTwitterアカウントの名前。
- twitterHandle  : チームのツイッターハンドル。

## seasons

In [15]:
ds.seasons.head().T

,0,1,2,3,4
seasonId,2017,2018,2019,2020,2021
seasonStartDate,2017-04-02,2018-03-29,2019-03-20,2020-07-23,2021-02-28
seasonEndDate,2017-11-01,2018-10-28,2019-10-30,2020-10-28,2021-10-31
preSeasonStartDate,2017-02-22,2018-02-21,2019-02-21,2020-02-21,2021-02-28
preSeasonEndDate,2017-04-01,2018-03-27,2019-03-26,2020-07-22,2021-03-30
regularSeasonStartDate,2017-04-02,2018-03-29,2019-03-20,2020-07-23,2021-04-01
regularSeasonEndDate,2017-10-01,2018-10-01,2019-09-29,2020-09-27,2021-10-03
lastDate1stHalf,2017-07-09,2018-07-15,2019-07-07,2020-08-25,2021-07-11
allStarDate,2017-07-11,2018-07-17,2019-07-09,NaN,2021-07-13
firstDate2ndHalf,2017-07-14,2018-07-19,2019-07-11,2020-08-26,2021-07-15


- seasonId : シーズンID
- seasonStartDate : シーズンスタート日
- seasonEndDate : シーズン終了日
- preSeasonStartDate : 1つ前のシーズンスタート日
- preSeasonEndDate : 1つ前のシーズンの終わりの日
- regularSeasonStartDate : レギュラーシーズンのスタートの日
- regularSeasonEndDate : レギュラーシーズンの終わりの日
- lastDate1stHalf : 1st halfの最終日
- allStarDate : オールスター戦の日付
- firstDate2ndHalf : 2nd halfの始まり日
- postSeasonStartDate : 次のシーズンのスタート日
- postSeasonEndDate : 次のシーズンの終わり日

## teams

In [16]:
ds.teams.head()

,id,name,teamName,teamCode,shortName,abbreviation,locationName,leagueId,leagueName,divisionId,divisionName,venueId,venueName
0,108,Los Angeles Angels,Angels,ana,LA Angels,LAA,Anaheim,103,American League,200,American League West,1,Angel Stadium
1,109,Arizona Diamondbacks,D-backs,ari,Arizona,ARI,Phoenix,104,National League,203,National League West,15,Chase Field
2,110,Baltimore Orioles,Orioles,bal,Baltimore,BAL,Baltimore,103,American League,201,American League East,2,Oriole Park at Camden Yards
3,111,Boston Red Sox,Red Sox,bos,Boston,BOS,Boston,103,American League,201,American League East,3,Fenway Park
4,112,Chicago Cubs,Cubs,chn,Chi Cubs,CHC,Chicago,104,National League,205,National League Central,17,Wrigley Field


- id : チームID
- name : 名前
- teamName : チームの名前
- teamCode : チームのコード
- shortName : 短い名前
- abbreviation : 略語
- locationName : 場所の名前
- leagueId : リーグのid
- leagueName : リーグの名前
- divisionId : 部門id
- divisionName : 部門名
- venueId : 会場id
- venueName : 会場名

## players

In [17]:
ds.players.head()

,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,665482,Gilberto Celestino,1999-02-13,2021-06-02,Santo Domingo,NaN,Dominican Republic,72,170,8,Outfielder,False
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True
2,661269,Vladimir Gutierrez,1995-09-18,2021-05-28,Havana,NaN,Cuba,73,190,1,Pitcher,True
3,669212,Eli Morgan,1996-05-13,2021-05-28,Rancho Palos Verdes,CA,USA,70,190,1,Pitcher,True
4,666201,Alek Manoah,1998-01-09,2021-05-27,Homestead,FL,USA,78,260,1,Pitcher,True


- playerId - Unique identifier for a player. : プレーヤーID-プレーヤーの一意の識別子。
- playerName : プレーヤの名前
- DOB - Player’s date of birth. : DOB-プレーヤーの生年月日。
- mlbDebutDate : MLBデビュー日
- birthCity : 生まれた町
- birthStateProvince : 出生州
- birthCountry : 生まれた国
- heightInches : 身長(inch)
- weight : 体重
- primaryPositionCode - Player’s primary position code : 主要ポジションコード
- primaryPositionName - player’s primary position : 主要ポジション名
- playerForTestSetAndFuturePreds - Boolean, true if player is among those for whom predictions are to be made in test data

## awards

In [18]:
ds.awards.head()

,awardDate,awardSeason,awardId,awardName,playerId,playerName,awardPlayerTeamId
0,2017-12-21,2017,WARRENSPAHN,Warren Spahn Award,477132,Clayton Kershaw,119.0
1,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,474319,Brandon Snyder,120.0
2,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,592530,Jose Marmolejos,120.0
3,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,593833,Wander Suero,120.0
4,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,600466,Raudy Read,120.0


このファイルには、日次データの開始前（つまり、2018年以前）にトレーニングセットのプレーヤーが獲得した賞が含まれています。
- awardDate - Date award was given. : 授与日 - 授与された日付。
- awardSeason - Season award was from. : アワードシーズン-シーズンアワードはからでした。
- awardId : アワードid
- awardName : アワード名
- playerId - Unique identifier for a player. : プレーヤーID-プレーヤーの一意の識別子。
- playerName : プレーヤーの名前
- awardPlayerTeamId : プレイヤーのチームID

## Target distribution

In [23]:
ds.train_next

,date,engagementMetricsDate,playerId,target1,target2,target3,target4
0,20180101,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294
1,20180101,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118
2,20180101,2018-01-02,519317,0.974327,56.177044,13.693746,64.166664
3,20180101,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745
4,20180101,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373
...,...,...,...,...,...,...,...
2506171,20210430,2021-05-01,451661,0.000000,0.013314,0.000000,0.625925
2506172,20210430,2021-05-01,519301,0.000131,0.003329,0.000000,0.216229
2506173,20210430,2021-05-01,527055,0.000000,0.019971,0.000000,0.273131
2506174,20210430,2021-05-01,543484,0.000131,0.056586,0.000000,1.024240
